<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Prediction_EN_791112_50000_51000_epoch5_040511.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Error: gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Ignore this error as gcsfs is not used.

In [1]:
!pip install -q accelerate
!pip install -q huggingface_hub
#!pip install -q scikit-learn
!pip install -q transformers datasets  # 2 Hugging Face libraries
#!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import shutil
import torch

from contextlib       import suppress
from datasets         import DatasetDict
from google.colab     import files, userdata
from huggingface_hub  import create_repo, hf_hub_download, login, whoami
from sklearn.metrics  import classification_report
from transformers     import LongformerTokenizerFast, LongformerForSequenceClassification
from torch.utils.data import DataLoader

## Hugging Face Hub (HF Hub) authenticate

In [3]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')

login(token=hf_token)

# Check
user = whoami(token=hf_token)
assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"

print(f"user: {user}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


user: {'type': 'user', 'id': '66ec3d5f61228b02f8780beb', 'name': 'claudelepere', 'fullname': 'Claude Lepère', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66ec3d5f61228b02f8780beb/gvnf9pvm2KvE90ETMUQo3.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'jobs_token', 'role': 'fineGrained', 'createdAt': '2025-01-04T17:44:35.493Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '66ec3d5f61228b02f8780beb', 'type': 'user', 'name': 'claudelepere'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


## HF Hub repo

In [4]:
repo_id = "claudelepere/jobs_EN_791112_50000_040511"

## Checks

In [5]:
!python -V

print(f"currentdir: {os.getcwd()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

Python 3.11.11
currentdir: /content
device: cuda


In [6]:
skills           = 791112
all_rows_low     = 50000
all_rows_high    = 51000
num_datapoints   = all_rows_high - all_rows_low
max_length       = 1024
batch_size       = 4*8
threshold_tuning = True

datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]

print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()


datasetDict_zip_file_name: dataset_EN_791112_50000_51000.zip
datasetDict_dir_name     : dataset_EN_791112_50000_51000



## Out Of Memory (OOM)

### OOM: check for and kill zombie processes

In [7]:
!ps aux | grep python
!kill -9 <PID>
if torch.cuda.is_available():
    !nvidia-smi
    print(torch.cuda.memory_summary())

root          85  2.0  0.0      0     0 ?        Z    19:58   0:13 [python3] <defunct>
root          86  0.0  0.0  77372 57768 ?        S    19:58   0:00 python3 /usr/local/bin/colab-file
root         131  0.6  0.1 793096 129944 ?       Sl   19:58   0:04 /usr/bin/python3 /usr/local/bin/j
root        2326 15.5  1.4 12322188 1284400 ?    Ssl  20:07   0:16 /usr/bin/python3 -m colab_kernel_
root        2363  0.5  0.0 545208 20424 ?        Sl   20:07   0:00 /usr/bin/python3 /usr/local/lib/p
root        2973  0.0  0.0   7376  3532 ?        S    20:09   0:00 /bin/bash -c ps aux | grep python
root        2975  0.0  0.0   6484  2224 ?        S    20:09   0:00 grep python
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `kill -9 <PID>'
Sun Apr  6 20:09:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----

### OOM: free up GPU memory

In [8]:
torch.cuda.empty_cache()

## Upload and unzip job dataset

In [9]:
def upload_unzip_dataset(filename):
    """Upload and unzip the dataset to /content, ensuring correct placement."""

    # Get the expected directory name (same as the zip filename without extension)
    expected_dir = os.path.splitext(filename)[0]

    # Check if the file and the directory exist in /content and delete them
    with suppress(FileNotFoundError):
        if os.path.isdir(expected_dir):
            shutil.rmtree(expected_dir)               # Remove directory if it exists
        if os.path.isfile(filename):
            os.remove(filename)                       # Remove file if it exists

    print(f"Removed '{expected_dir}' and '{filename}' if they were present in /content.")

    # Upload the zip file
    uploaded_files = files.upload()                  # Prompt file upload dialog

    if filename not in uploaded_files:
        raise FileNotFoundError(f"'{filename}' was not uploaded.")

    print(f"'{filename}' successfully uploaded to /content.")

    # Unzip the file to /content
    shutil.unpack_archive(filename, "/content")

    print(f"Unzipped to '/content/{expected_dir}'.")

# Usage
upload_unzip_dataset(datasetDict_zip_file_name)

Removed 'dataset_EN_791112_50000_51000' and 'dataset_EN_791112_50000_51000.zip' if they were present in /content.


Saving dataset_EN_791112_50000_51000.zip to dataset_EN_791112_50000_51000.zip
'dataset_EN_791112_50000_51000.zip' successfully uploaded to /content.
Unzipped to '/content/dataset_EN_791112_50000_51000'.


## Create datasetDict (HF DatasetDict) = 1 HF Dataset prediction
The sequence of datapoints of dataset=datasetDict['prediction'] is the same as the sequence of SELECT, id, ... WHERE in_LanguageId=1 ORDER BY id DESC in MySQL.

In [10]:
datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)
dataset     = datasetDict['prediction']

print(f"dataset: {type(dataset)} shape={dataset.shape}\n{dataset}")     # <class 'datasets.arrow_dataset.Dataset'> shape=(100, 8)
print(f"dataset.features: {type(dataset.features)} shape={dataset.features}\n{dataset.features}")

# Convert the dataset to a pandas dataframe
df_original = pd.DataFrame(dataset)
print()
print(f"df_original: {type(df_original)} shape={df_original.shape}\n{df_original}") # <class 'pandas.core.frame.DataFrame'> shape=(1000, 152)                     # <class 'pandas.core.frame.DataFrame'> shape=(100, 8)


dataset: <class 'datasets.arrow_dataset.Dataset'> shape=(1000, 152)
Dataset({
    features: ['id', 'text', '610', '611', '613', '614', '615', '618', '619', '637', '142', '143', '146', '147', '149', '150', '152', '154', '155', '156', '667', '158', '668', '160', '670', '162', '672', '673', '165', '676', '167', '677', '169', '170', '171', '678', '173', '174', '175', '176', '686', '687', '689', '683', '756', '758', '760', '761', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301', '302', '303', '304', '810', '811', '812', '813', '814', '815', '816', '817', '818', '819', '820', '821', '822', '823', '824', '825', '826', '827', '828', '829', '830', '681', '356', '358', '359', '360', '361', '362', '682', '364', '365', '367', '369', '370', '371', '373', '375', '376', '379', '382', '385', '900', '901', '902', '903', '904', '905', '394', '906

## Create labels (list), id2label (dict) and label2id (dict).
**The sequence of the labels list is the same as in dataset.
And the sequences of the optimized thresholds, true labels and predictions are preserved.**

* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [11]:
labels = [label for label in dataset.features.keys() if label not in ['id', 'text']]
#labels.sort()

print(f"labels: {type(labels)} {len(labels)}\n{labels}")

num_labels = len(labels)

labels: <class 'list'> 150
['610', '611', '613', '614', '615', '618', '619', '637', '142', '143', '146', '147', '149', '150', '152', '154', '155', '156', '667', '158', '668', '160', '670', '162', '672', '673', '165', '676', '167', '677', '169', '170', '171', '678', '173', '174', '175', '176', '686', '687', '689', '683', '756', '758', '760', '761', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301', '302', '303', '304', '810', '811', '812', '813', '814', '815', '816', '817', '818', '819', '820', '821', '822', '823', '824', '825', '826', '827', '828', '829', '830', '681', '356', '358', '359', '360', '361', '362', '682', '364', '365', '367', '369', '370', '371', '373', '375', '376', '379', '382', '385', '900', '901', '902', '903', '904', '905', '394', '906', '907', '908', '909', '910', '911', '912', '914', '913', '916', '915', '408',

In [12]:
id2label = {idx: label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

id2label: <class 'dict'> 150
{0: '610', 1: '611', 2: '613', 3: '614', 4: '615', 5: '618', 6: '619', 7: '637', 8: '142', 9: '143', 10: '146', 11: '147', 12: '149', 13: '150', 14: '152', 15: '154', 16: '155', 17: '156', 18: '667', 19: '158', 20: '668', 21: '160', 22: '670', 23: '162', 24: '672', 25: '673', 26: '165', 27: '676', 28: '167', 29: '677', 30: '169', 31: '170', 32: '171', 33: '678', 34: '173', 35: '174', 36: '175', 37: '176', 38: '686', 39: '687', 40: '689', 41: '683', 42: '756', 43: '758', 44: '760', 45: '761', 46: '263', 47: '265', 48: '266', 49: '267', 50: '268', 51: '270', 52: '271', 53: '272', 54: '273', 55: '274', 56: '275', 57: '276', 58: '277', 59: '278', 60: '279', 61: '283', 62: '284', 63: '285', 64: '286', 65: '287', 66: '288', 67: '289', 68: '291', 69: '292', 70: '293', 71: '297', 72: '298', 73: '299', 74: '300', 75: '301', 76: '302', 77: '303', 78: '304', 79: '810', 80: '811', 81: '812', 82: '813', 83: '814', 84: '815', 85: '816', 86: '817', 87: '818', 88: '819', 8

In [13]:
label2id = {label: idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

label2id: <class 'dict'> 150
{'610': 0, '611': 1, '613': 2, '614': 3, '615': 4, '618': 5, '619': 6, '637': 7, '142': 8, '143': 9, '146': 10, '147': 11, '149': 12, '150': 13, '152': 14, '154': 15, '155': 16, '156': 17, '667': 18, '158': 19, '668': 20, '160': 21, '670': 22, '162': 23, '672': 24, '673': 25, '165': 26, '676': 27, '167': 28, '677': 29, '169': 30, '170': 31, '171': 32, '678': 33, '173': 34, '174': 35, '175': 36, '176': 37, '686': 38, '687': 39, '689': 40, '683': 41, '756': 42, '758': 43, '760': 44, '761': 45, '263': 46, '265': 47, '266': 48, '267': 49, '268': 50, '270': 51, '271': 52, '272': 53, '273': 54, '274': 55, '275': 56, '276': 57, '277': 58, '278': 59, '279': 60, '283': 61, '284': 62, '285': 63, '286': 64, '287': 65, '288': 66, '289': 67, '291': 68, '292': 69, '293': 70, '297': 71, '298': 72, '299': 73, '300': 74, '301': 75, '302': 76, '303': 77, '304': 78, '810': 79, '811': 80, '812': 81, '813': 82, '814': 83, '815': 84, '816': 85, '817': 86, '818': 87, '819': 88, '

In [14]:
indices_under_900 = [i for i, label in enumerate(labels) if int(label) < 900]
print(f"indices_under_900: {type(indices_under_900)} len={len(indices_under_900)}\n{indices_under_900}")
#raise Exception()

indices_under_900: <class 'list'> len=133
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 126, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]


## Download the tokenizer and the model

In [15]:
print("Tokenizer")
tokenizer = LongformerTokenizerFast.from_pretrained(repo_id, timeout=60)  # Increased timeout to 60 seconds)

print("Model")
model = LongformerForSequenceClassification.from_pretrained(repo_id)
model.eval()  # Ensures no gradient computation

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Tokenizer


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Model


config.json:   0%|          | 0.00/5.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

## Downloads the optimized thresholds

In [16]:
file_path = hf_hub_download(repo_id=repo_id, filename="optimized_thresholds.json", cache_dir="./my_HF_downloads")
print(f"file_path: {file_path}")

# Load the JSON file
with open(file_path, 'r') as json_file:
    optimized_thresholds_json = json.load(json_file)

#print(f"optimized_thresholds_json: {type(optimized_thresholds_json)} {len(optimized_thresholds_json)}")
#print(json.dumps(optimized_thresholds_json, indent=2))

optimized_thresholds = np.array(list(optimized_thresholds_json.values()))
print(f"optimized_thresholds: {type(optimized_thresholds)} shape={optimized_thresholds.shape}\n{optimized_thresholds}")


optimized_thresholds.json:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

file_path: ./my_HF_downloads/models--claudelepere--jobs_EN_791112_50000_040511/snapshots/2c14217bbecc880b364665fc7b0c35a485b93f93/optimized_thresholds.json
optimized_thresholds: <class 'numpy.ndarray'> shape=(150,)
[0.5  0.4  0.45 0.45 0.45 0.4  0.3  0.45 0.5  0.4  0.45 0.45 0.45 0.5
 0.4  0.4  0.45 0.5  0.4  0.45 0.4  0.45 0.4  0.5  0.25 0.2  0.5  0.45
 0.45 0.45 0.45 0.45 0.45 0.25 0.4  0.45 0.45 0.55 0.45 0.4  0.5  0.45
 0.45 0.45 0.5  0.45 0.3  0.4  0.35 0.35 0.35 0.4  0.45 0.45 0.4  0.45
 0.35 0.45 0.45 0.4  0.4  0.35 0.45 0.35 0.45 0.4  0.4  0.45 0.45 0.35
 0.4  0.45 0.45 0.35 0.45 0.4  0.45 0.4  0.45 0.35 0.45 0.4  0.45 0.45
 0.4  0.45 0.45 0.5  0.4  0.4  0.45 0.45 0.45 0.45 0.4  0.4  0.45 0.35
 0.45 0.5  0.3  0.45 0.45 0.45 0.45 0.45 0.4  0.35 0.45 0.45 0.45 0.2
 0.4  0.45 0.4  0.45 0.4  0.4  0.35 0.45 0.05 0.05 0.05 0.1  0.05 0.05
 0.45 0.4  0.05 0.15 0.05 0.05 0.2  0.05 0.05 0.1  0.05 0.05 0.5  0.45
 0.45 0.35 0.35 0.45 0.45 0.45 0.45 0.5  0.5  0.5 ]


In [17]:
# Step 1: Tokenize dataset
def tokenize_function(examples):
  return tokenizer(examples["text"], padding='max_length', max_length=max_length, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
# <class 'datasets.arrow_dataset.Dataset'> shape=(100, 10)
#print(f"tokenized_dataset: {type(tokenized_dataset)} shape={tokenized_dataset.shape}\n{tokenized_dataset}")

tokenized_df = tokenized_dataset.to_pandas()
#print(f"tokenized_df: {type(tokenized_df)} shape={tokenized_df.shape}\n{tokenized_df}")

# Step 2: Convert to PyTorch DataLoader
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"] + labels)
dataloader = DataLoader(tokenized_dataset, batch_size=batch_size)

# Step 3: Run Prediction
all_preds = []

with torch.no_grad():                       # No gradients needed for prediction
    for batch in dataloader:
        # Move batch to GPU if available
        inputs = {k: batch[k].to(device) for k in ["input_ids", "attention_mask"]}

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits  # Model outputs raw logits

        # Apply sigmoid to convert logits into probabilities
        probs = torch.sigmoid(logits)

        # Move probs to CPU and convert to NumPy
        all_preds.extend(probs.cpu().numpy())

all_preds_arr = np.array(all_preds)
#print(f"all_preds    : {type(all_preds)}     len={len(all_preds)}\n{all_preds}")
#print(f"all_preds_arr: {type(all_preds_arr)} shape={all_preds_arr.shape}\n{all_preds_arr}")

# Step 4: Convert Probabilities to Binary Predictions
if threshold_tuning:
    thresholds = optimized_thresholds
else:
    thresholds = np.full(num_labels, 0.5)
print(f"thresholds: {type(thresholds)} shape={thresholds.shape}\n{thresholds}")                  # <class 'numpy.ndarray'> shape=(6,)

binary_preds = (np.array(all_preds) > thresholds).astype(int)
#print(f"binary_preds: {type(binary_preds)} shape={binary_preds.shape}\n{binary_preds}")          # <class 'numpy.ndarray'> shape=(100, 6)

# Step 5: Compare with True Labels
true_labels = tokenized_df[labels]
#print(f"true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")              # <class 'pandas.core.frame.DataFrame'> shape=(100, 6)

# Convert true_labels DataFrame to a NumPy array of 0 and 1
true_labels_np  = true_labels.astype(int).to_numpy()
#print(f"true_labels_np: {type(true_labels_np)} shape={true_labels_np.shape}\n{true_labels_np}")  # <class 'numpy.ndarray'> shape=(100, 6)

# Convert id2label dict to a list of label names
label_names = list(id2label.values())
print(f"label_names: {type(label_names)} len={len(label_names)}\n{label_names}")                 # <class 'list'> len=6 ['390', '135', '136', '137', '138', '139']
report = classification_report(true_labels_np, binary_preds, target_names=label_names, zero_division=0)

print(f"classification report:{type(report)} len={len(report)}\n{report}")

df_compare                      = df_original.copy()           # <class 'pandas.core.frame.DataFrame'> shape=(1000, 152)
df_compare['true_labels']       = true_labels.values.tolist()  # <class 'pandas.core.series.Series'> shape=(1000,)
df_compare['preds']             = binary_preds.tolist()
df_compare['compare']           = (df_compare['true_labels']==df_compare['preds']).replace({True:'OK', False:'KO'})
df_compare['count_true_labels'] = df_compare['true_labels'].apply(lambda x: sum(x))
df_compare['count_preds']       = df_compare['preds'].apply(lambda x: sum(x))

#indices_under_900 = [i for i, label in enumerate(labels) if int(label) < 900]
#print(f"indices_under_900: {type(indices_under_900)} len={len(indices_under_900)}\n{indices_under_900}")

#def count_1s_under_900(label_list):
#    return sum(label_list[i] for i in indices_under_900) # if i < len(label_list))

#df_compare['count_true_labels_under_900'] = df_compare['true_labels'].apply(count_1s_under_900)
#df_compare['count_preds_under_900']       = df_compare['preds'].apply(count_1s_under_900)

def mask_preds_under_900(preds, valid_indices):
    masked = np.zeros_like(preds)
    # Convert valid_indices to NumPy array to enable advanced indexing
    valid_indices_np = np.array(valid_indices)
    masked[valid_indices_np] = np.array(preds)[valid_indices_np]
    return masked.tolist()

df_compare['true_labels_under900']       = df_compare['true_labels'].apply(lambda row: mask_preds_under_900(row, indices_under_900))
df_compare['preds_under900']             = df_compare['preds'].apply(lambda row: mask_preds_under_900(row, indices_under_900))
df_compare['compare_under900']           = (df_compare['true_labels_under900']==df_compare['preds_under900']).replace({True:'OK', False:'KO'})
df_compare['count_true_labels_under900'] = df_compare['true_labels_under900'].apply(lambda x: sum(x))
df_compare['count_preds_under900']       = df_compare['preds_under900'].apply(lambda x: sum(x))
#print(f"df_compare: {type(df_compare)} shape={df_compare.shape}\n{df_compare}")                  # <class 'pandas.core.frame.DataFrame'> shape=(100, 10)
#print(df_compare.head(2))
print()
#raise Exception()

def print_row(row, file):
    """Prints the row information to the console and writes it to a file."""
    output  = f"id: {row['id']}\n"
    output += f"text: {row['text']}\n"
    output += "labels     : [" + ", ".join(f"{label:>5}" for label in labels) + "]\n"
    output += "true_labels: [" + ", ".join(f"{true_label:>5}" for true_label in row['true_labels']) + "]\n"
    output += "preds      : [" + ", ".join(f"{pred:>5}" for pred in row['preds']) + "]\n"
    output += f"compare   : {row['compare']}\n"
    output += f"count_true_labels: {row['count_true_labels']}\n"
    output += f"count_preds      : {row['count_preds']}\n"

    output += "labels              : [" + ", ".join(f"{label:>5}" for label in labels) + "]\n"
    output += "true_labels_under900: [" + ", ".join(f"{true_label:>5}" for true_label in row['true_labels_under900']) + "]\n"
    output += "preds_under900      : [" + ", ".join(f"{pred:>5}" for pred in row['preds_under900']) + "]\n"
    output += f"compare_under900   : {row['compare_under900']}\n"
    #output += f"count_true_labels_under_900: {row['count_true_labels_under_900']}\n"
    #output += f"count_preds_under_900      : {row['count_preds_under_900']}\n"
    output += f"count_true_labels_under900: {row['count_true_labels_under900']}\n"
    output += f"count_preds_under900      : {row['count_preds_under900']}\n"
    output += "\n"

    print(output, end="")  # Print to console without extra newline
    file.write(output)     # Write to file


# Open the file in write mode
with open("output.txt", "w") as f:
    _ = df_compare.apply(print_row, axis=1, args=(f,))  # Pass file object to print_row

files.download("/content/output.txt")

# Count total 'OK' and 'KO' values
total_ok = (df_compare['compare'] == 'OK').sum()  # Total matches
total_ko = (df_compare['compare'] == 'KO').sum()  # Total mismatches
print(f"Total 'OK': {total_ok}")
print(f"Total 'KO': {total_ko}")

total_ok_under900 = (df_compare['compare_under900'] == 'OK').sum()  # Total matches
total_ko_under900 = (df_compare['compare_under900'] == 'KO').sum()  # Total mismatches
print(f"Total 'OK_under900': {total_ok_under900}")
print(f"Total 'KO_under900': {total_ko_under900}")
print()

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Initializing global attention on CLS token...


Streaming output truncated to the last 5000 lines.
preds_under900      : [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     1,     0,     0,     0,     0,     0,     1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Total 'OK': 0
Total 'KO': 1000
Total 'OK_under900': 40
Total 'KO_under900': 960



In [18]:
print("It's the end")

It's the end


In [19]:
raise Exception("I stop here")

Exception: I stop here